In [34]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
import warnings

<h2> Get Working Directory and Join CSV file for Pandas </h2>

In [3]:
pwd

'C:\\Users\\clyde\\Documents\\Thinkful\\Data Science Bootcamp\\Unit-3\\Lesson 5'

In [138]:
os.listdir()[12]

'Melbourne_housing_FULL.csv'

In [139]:
path = os.path.join(os.getcwd(),os.listdir()[12])

<h2> Read CSV File </h2>

In [140]:
df = pd.read_csv(path)

<h2> Examine Data Head </h2>

In [141]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


<h2> Check DataFrame Columns </h2>

In [142]:
df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

<h2> Check DataFrame Data Types </h2>

In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
Suburb           34857 non-null object
Address          34857 non-null object
Rooms            34857 non-null int64
Type             34857 non-null object
Price            27247 non-null float64
Method           34857 non-null object
SellerG          34857 non-null object
Date             34857 non-null object
Distance         34856 non-null float64
Postcode         34856 non-null float64
Bedroom2         26640 non-null float64
Bathroom         26631 non-null float64
Car              26129 non-null float64
Landsize         23047 non-null float64
BuildingArea     13742 non-null float64
YearBuilt        15551 non-null float64
CouncilArea      34854 non-null object
Lattitude        26881 non-null float64
Longtitude       26881 non-null float64
Regionname       34854 non-null object
Propertycount    34854 non-null float64
dtypes: float64(12), int64(1), object(8)
memory usage: 5.6+ M

<h2> Check Data Fill Factor </h2>

In [148]:
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,BuildingArea,21115,39.423932
1,YearBuilt,19306,44.613707
2,Landsize,11810,66.118714
3,Car,8728,74.960553
4,Bathroom,8226,76.400723
5,Bedroom2,8217,76.426543
6,Longtitude,7976,77.117939
7,Lattitude,7976,77.117939
8,Price,7610,78.167943
9,CouncilArea,3,99.991393


<h1> Impute missing values </h1>

In [164]:
df_new = df.select_dtypes(exclude=['object'])

<h3> Exclude Object Data Type </h3>

In [169]:
df.select_dtypes(exclude=['object']).columns

Index(['Rooms', 'Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude',
       'Propertycount'],
      dtype='object')

In [165]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
df_new = my_imputer.fit_transform(df_new)

<h3> Convert Numpy Array back to DataFrame </h3>

In [174]:
df_new = pd.DataFrame(data=df_new,    # values
...              index=[np.arange(0,len(df_new))],    # 1st column as index
...              columns=df.select_dtypes(exclude=['object']).columns)  # 1st row as the column names
df_new.head()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2.0,1.050173e+06,2.5,3067.0,2.0,1.0,1.0,126.0,160.2564,1965.289885,-37.8014,144.9958,4019.0
1,2.0,1.480000e+06,2.5,3067.0,2.0,1.0,1.0,202.0,160.2564,1965.289885,-37.7996,144.9984,4019.0
2,2.0,1.035000e+06,2.5,3067.0,2.0,1.0,0.0,156.0,79.0000,1900.000000,-37.8079,144.9934,4019.0
3,3.0,1.050173e+06,2.5,3067.0,3.0,2.0,1.0,0.0,160.2564,1965.289885,-37.8114,145.0116,4019.0
4,3.0,1.465000e+06,2.5,3067.0,3.0,2.0,0.0,134.0,150.0000,1900.000000,-37.8093,144.9944,4019.0


In [187]:
df.select_dtypes('object').head()

,Suburb,Address,Type,Method,SellerG,Date,CouncilArea,Regionname
0,Abbotsford,68 Studley St,h,SS,Jellis,3/09/2016,Yarra City Council,Northern Metropolitan
1,Abbotsford,85 Turner St,h,S,Biggin,3/12/2016,Yarra City Council,Northern Metropolitan
2,Abbotsford,25 Bloomburg St,h,S,Biggin,4/02/2016,Yarra City Council,Northern Metropolitan
3,Abbotsford,18/659 Victoria St,u,VB,Rounds,4/02/2016,Yarra City Council,Northern Metropolitan
4,Abbotsford,5 Charles St,h,SP,Biggin,4/03/2017,Yarra City Council,Northern Metropolitan


In [185]:
pd.concat([df.select_dtypes('object'),df_new],axis=1,sort=True)


TypeError: object of type 'int' has no len()

In [193]:
df_new.merge(df.select_dtypes('object'),left_index=True,right_index=True)

MergeError: Must pass right_on or right_index=True

In [145]:
dummy = pd.get_dummies(df['Suburb'])

In [146]:
dummy.head

<bound method NDFrame.head of        Abbotsford  Aberfeldie  Airport West  Albanvale  Albert Park  Albion  \
0               1           0             0          0            0       0   
1               1           0             0          0            0       0   
2               1           0             0          0            0       0   
3               1           0             0          0            0       0   
4               1           0             0          0            0       0   
5               1           0             0          0            0       0   
6               1           0             0          0            0       0   
7               1           0             0          0            0       0   
8               1           0             0          0            0       0   
9               1           0             0          0            0       0   
10              1           0             0          0            0       0   
11              1     

In [29]:
dummy.columns

Index(['Abbotsford', 'Aberfeldie', 'Airport West', 'Albanvale', 'Albert Park',
       'Albion', 'Alphington', 'Altona', 'Altona Meadows', 'Altona North',
       ...
       'Woori Yallock', 'Wyndham Vale', 'Yallambie', 'Yan Yean', 'Yarra Glen',
       'Yarra Junction', 'Yarrambat', 'Yarraville', 'croydon', 'viewbank'],
      dtype='object', length=380)

In [147]:
df_subdrop = df.drop('Suburb',axis=1)

In [31]:
df_mergedummy = pd.concat([df_subdrop,dummy],axis=1)

In [32]:
df_mergedummy.head()

,Address,Rooms,Type,Price,Method,SellerG,Date,Postcode,Regionname,Propertycount,...,Woori Yallock,Wyndham Vale,Yallambie,Yan Yean,Yarra Glen,Yarra Junction,Yarrambat,Yarraville,croydon,viewbank
0,49 Lithgow St,3,h,1490000.0,S,Jellis,1/04/2017,3067,Northern Metropolitan,4019,...,0,0,0,0,0,0,0,0,0,0
1,59A Turner St,3,h,1220000.0,S,Marshall,1/04/2017,3067,Northern Metropolitan,4019,...,0,0,0,0,0,0,0,0,0,0
2,119B Yarra St,3,h,1420000.0,S,Nelson,1/04/2017,3067,Northern Metropolitan,4019,...,0,0,0,0,0,0,0,0,0,0
3,68 Vida St,3,h,1515000.0,S,Barry,1/04/2017,3040,Western Metropolitan,1543,...,0,0,0,0,0,0,0,0,0,0
4,92 Clydesdale Rd,2,h,670000.0,S,Nelson,1/04/2017,3042,Western Metropolitan,3464,...,0,0,0,0,0,0,0,0,0,0


In [33]:
df_mergedummy.shape

(63023, 392)

In [86]:
price_corr = df_mergedummy.corrwith(df_mergedummy.Price)
price_corr

Rooms                 0.412438
Price                 1.000000
Postcode              0.003112
Propertycount        -0.060769
Distance             -0.253668
Abbotsford            0.003865
Aberfeldie            0.025282
Airport West         -0.026058
Albanvale            -0.016913
Albert Park           0.083428
Albion               -0.027495
Alphington            0.026382
Altona               -0.010159
Altona Meadows       -0.022948
Altona North         -0.020678
Ardeer               -0.028474
Armadale              0.062244
Ascot Vale            0.009004
Ashburton             0.061343
Ashwood               0.015454
Aspendale             0.005843
Aspendale Gardens    -0.007207
Attwood              -0.012573
Avondale Heights     -0.016650
Avonsleigh                 NaN
Bacchus Marsh        -0.012478
Balaclava            -0.012892
Balwyn                0.118711
Balwyn North          0.114683
Bayswater            -0.017274
                        ...   
Warrandyte South      0.002925
Warranwo

In [52]:
price_corr_index = price_corr[price_corr.isnull()].index

In [78]:
df_mergedummydropna = df_mergedummy.drop(price_corr_index,axis=1)
df_mergedummydropna.shape

(63023, 382)

In [76]:
df_mergedummydropna.dropna().shape

(48433, 382)

In [85]:
df_mergedummydropna.loc[:,df_mergedummydropna.columns[10:20]]

,Distance,CouncilArea,Abbotsford,Aberfeldie,Airport West,Albanvale,Albert Park,Albion,Alphington,Altona
0,3.0,Yarra City Council,1,0,0,0,0,0,0,0
1,3.0,Yarra City Council,1,0,0,0,0,0,0,0
2,3.0,Yarra City Council,1,0,0,0,0,0,0,0
3,7.5,Moonee Valley City Council,0,1,0,0,0,0,0,0
4,10.4,Moonee Valley City Council,0,0,1,0,0,0,0,0
5,10.4,Moonee Valley City Council,0,0,1,0,0,0,0,0
6,10.4,Moonee Valley City Council,0,0,1,0,0,0,0,0
7,10.4,Moonee Valley City Council,0,0,1,0,0,0,0,0
8,14.0,Brimbank City Council,0,0,0,1,0,0,0,0
9,3.0,Port Phillip City Council,0,0,0,0,1,0,0,0


In [93]:
price_corr1 = df_mergedummydropna.corrwith(df_mergedummydropna.Price)
price_corr1[price_corr1<-.5]

Series([], dtype: float64)

In [123]:
price_corr1.index[1:20]

Index(['Price', 'Postcode', 'Propertycount', 'Distance', 'Abbotsford',
       'Aberfeldie', 'Airport West', 'Albanvale', 'Albert Park', 'Albion',
       'Alphington', 'Altona', 'Altona Meadows', 'Altona North', 'Ardeer',
       'Armadale', 'Ascot Vale', 'Ashburton', 'Ashwood'],
      dtype='object')

In [99]:
df_mergedummydropna.Type.unique()

array(['h', 't', 'u'], dtype=object)

In [125]:
df_dummytype = pd.get_dummies(df_mergedummydropna.Type)

In [126]:
df_mergedummydropna = df_mergedummydropna.drop('Type',axis=1)

In [127]:
df_typesubdummy = pd.concat([df_mergedummydropna,df_dummytype],axis=1)

In [128]:
price_corr2 = df_typesubdummy.corrwith(df_typesubdummy.Price)

In [132]:
price_corr2['Propertycount']

-0.0607693339369981

In [133]:
df.Method.unique()

array(['S', 'SP', 'PI', 'SN', 'VB', 'PN', 'SA', 'W', 'SS'], dtype=object)